<a href="https://colab.research.google.com/github/amfei/Statistical_Inference/blob/main/causal_inference2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Effect of a featute with 2 categories(yes /no)  on Previous Claims
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ttest_ind
from statsmodels.stats.power import TTestIndPower
from sklearn.preprocessing import StandardScaler

def causal_effect_estimation_SmokingStatus(df, treatment_col, outcome_col):
    # Drop rows with missing values in relevant columns
    df_clean = df.dropna().copy()


    # Clean string and map 'Yes'/'No' to 1/0 if needed
    if df_clean[treatment_col].dtype == object:
        df_clean[treatment_col] = df_clean[treatment_col].astype(str).str.strip()
        df_clean[treatment_col] = df_clean[treatment_col].map({'Yes': 1, 'No': 0})

    # Ensure binary treatment column
    if df_clean[treatment_col].nunique() != 2:
        raise ValueError("This function only supports binary treatment columns (e.g., 'Yes'/'No').")

    # One-hot encode object columns (except treatment and outcome)
    categorical_cols = df_clean.select_dtypes(include='object').columns.difference([treatment_col, outcome_col])
    df_encoded = pd.get_dummies(df_clean, columns=categorical_cols, drop_first=True)

    # Convert boolean to int
    bool_cols = df_encoded.select_dtypes(include='bool').columns
    df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)

    # Define confounders
    confounders = df_encoded.columns.difference([treatment_col, outcome_col])
    X = df_encoded[confounders]
    T = df_encoded[treatment_col]

    # Standardize for logistic regression stability
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Propensity score model
    ps_model = LogisticRegression(solver='lbfgs', max_iter=1000)
    ps_model.fit(X_scaled, T)
    df_encoded['propensity_score'] = ps_model.predict_proba(X_scaled)[:, 1]

    # Split into treated and control groups
    treated = df_encoded[df_encoded[treatment_col] == 1]
    control = df_encoded[df_encoded[treatment_col] == 0]

    # Nearest neighbor matching on propensity score
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control[['propensity_score']])
    distances, indices = nn.kneighbors(treated[['propensity_score']])
    matched_control = control.iloc[indices.flatten()].reset_index(drop=True)
    matched_treated = treated.reset_index(drop=True)

    # Extract outcomes
    treated_outcomes = matched_treated[outcome_col]
    control_outcomes = matched_control[outcome_col]

    # Estimate ATE and other statistics
    ate = treated_outcomes.mean() - control_outcomes.mean()
    t_stat, p_val = ttest_ind(treated_outcomes, control_outcomes)

    # Cohen's d
    s_pooled = np.sqrt(
        ((len(treated_outcomes) - 1) * treated_outcomes.var() +
         (len(control_outcomes) - 1) * control_outcomes.var()) /
        (len(treated_outcomes) + len(control_outcomes) - 2)
    )
    cohens_d = ate / s_pooled

    # Required sample size for 80% power
    analysis = TTestIndPower()
    required_n = analysis.solve_power(effect_size=abs(cohens_d), alpha=0.05, power=0.8, ratio=1)

    # Return summary as DataFrame
    summary_results = pd.DataFrame({
        "Metric": ["ATE", "T-Statistic", "P-Value", "Cohen's d", "Required Sample Size"],
        "Value": [ate, t_stat, p_val, cohens_d, required_n]
    })

    return summary_results

causal_effect_estimation_SmokingStatus(df, treatment_col='Smoking Status', outcome_col='Previous Claims')

In [ ]:
columns = ['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Premium Amount',
       'Policy Start Date', 'Customer Feedback', 'Smoking Status',
       'Exercise Frequency', 'Property Type']

In [ ]:
# Effect of multi categories(>2) featutes on Previous Claims

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ttest_ind
from statsmodels.stats.power import TTestIndPower
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("InsuranceDataset.csv")

df.drop('Policy Start Date', axis = 1, inplace = True)

def causal_effect_estimation(df, treatment_col, outcome_col):
    # Drop rows with missing values in relevant columns
    df_clean = df.dropna()

    # If binary Yes/No, map to 1/0
    if df_clean[treatment_col].dtype == object:
        unique_vals = df_clean[treatment_col].dropna().unique()
        if set(unique_vals) == {'Yes', 'No'}:
            df_clean.loc[:, treatment_col] = df_clean[treatment_col].map({'Yes': 1, 'No': 0})
        else:
            # Automatically factorize any non-binary, non-Yes/No column
            df_clean[treatment_col], _ = pd.factorize(df_clean[treatment_col])

    # One-hot encode remaining object columns (excluding treatment and outcome)
    categorical_cols = df_clean.select_dtypes(include='object').columns.difference([treatment_col, outcome_col])
    df_encoded = pd.get_dummies(df_clean, columns=categorical_cols, drop_first=True)

    # Convert boolean to int
    df_encoded[df_encoded.select_dtypes(include='bool').columns] = df_encoded.select_dtypes(include='bool').astype(int)

    # Define confounders
    confounders = df_encoded.columns.difference([treatment_col, outcome_col])

    # Propensity Score Estimation
    X_ps = df_encoded[confounders]
    T = df_encoded[treatment_col]

    ps_model = LogisticRegression(solver='lbfgs', max_iter=500)
    ps_model.fit(X_ps, T)

    # If binary, select prob for class 1; else take max prob
    proba = ps_model.predict_proba(X_ps)
    if proba.shape[1] == 2:
        df_encoded['propensity_score'] = proba[:, 1]
    else:
        df_encoded['propensity_score'] = proba.max(axis=1)

    # Define treated and control for one-vs-all
    treated = df_encoded[df_encoded[treatment_col] == df_encoded[treatment_col].unique()[0]]
    control = df_encoded[df_encoded[treatment_col] != df_encoded[treatment_col].unique()[0]]

    # Nearest neighbor matching
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control[['propensity_score']])
    distances, indices = nn.kneighbors(treated[['propensity_score']])
    matched_control = control.iloc[indices.flatten()].reset_index(drop=True)
    matched_treated = treated.reset_index(drop=True)

    # Extract outcomes
    treated_outcomes = matched_treated[outcome_col]
    control_outcomes = matched_control[outcome_col]

    # ATE
    ate = treated_outcomes.mean() - control_outcomes.mean()

    # T-test
    t_stat, p_val = ttest_ind(treated_outcomes, control_outcomes)

    # Cohen's d
    s_pooled = np.sqrt(((len(treated_outcomes) - 1) * treated_outcomes.var() +
                        (len(control_outcomes) - 1) * control_outcomes.var()) /
                        (len(treated_outcomes) + len(control_outcomes) - 2))
    cohens_d = ate / s_pooled

    # Required Sample Size
    analysis = TTestIndPower()
    required_n = analysis.solve_power(effect_size=abs(cohens_d), alpha=0.05, power=0.8, ratio=1)

    # Result summary
    summary_results = pd.DataFrame({
        "Metric": ["ATE", "T-Statistic", "P-Value", "Cohen's d", "Required Sample Size"],
        "Value": [ate, t_stat, p_val, cohens_d, required_n]
    })

    return summary_results

causal_effect_estimation(df, treatment_col='Property Type', outcome_col='Previous Claims')